# Build a Contextual Retrieval based RAG System

## Install OpenAI, and LangChain dependencies

In [18]:
!pip install langchain==0.3.26
!pip install langchain-openai==0.3.28
!pip install langchain-community==0.3.27
!pip install jq==1.9.1
!pip install pymupdf==1.26.3

## Install Chroma Vector DB and LangChain wrapper

In [19]:
!pip install langchain-chroma==0.2.4

## Enter Open AI API Key

In [20]:
n
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Setup Environment Variables

In [21]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [22]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Loading and Processing the Data

### Get the dataset

In [23]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1aZxZejfteVuofISodUrY2CDoyuPLYDGZ download it
# manually upload it on colab
!gdown 1aZxZejfteVuofISodUrY2CDoyuPLYDGZ

Downloading...
From: https://drive.google.com/uc?id=1aZxZejfteVuofISodUrY2CDoyuPLYDGZ
To: /content/rag_docs.zip
100% 5.92M/5.92M [00:00<00:00, 39.3MB/s]


In [24]:
!unzip rag_docs.zip

Archive:  rag_docs.zip
replace rag_docs/attention_paper.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rag_docs/cnn_paper.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rag_docs/resnet_paper.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rag_docs/vision_transformer.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rag_docs/wikidata_rag_demo.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


### Load and Process JSON Documents

In [25]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='./rag_docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
wiki_docs = loader.load()

In [26]:
len(wiki_docs)

1801

In [27]:
wiki_docs[3]

Document(metadata={'source': '/content/rag_docs/wikidata_rag_demo.jsonl', 'seq_num': 4}, page_content='{"id": "71548", "title": "Chi-square distribution", "paragraphs": ["In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\\u00a0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution.", "Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other."]}')

In [28]:
import json
from langchain.docstore.document import Document
wiki_docs_processed = []

for doc in wiki_docs:
    doc = json.loads(doc.page_content)
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
        "source": "Wikipedia",
        "page": 1
    }
    data = ' '.join(doc['paragraphs'])
    wiki_docs_processed.append(Document(page_content=data, metadata=metadata))

In [29]:
wiki_docs_processed[3]

Document(metadata={'title': 'Chi-square distribution', 'id': '71548', 'source': 'Wikipedia', 'page': 1}, page_content='In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\xa0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution. Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other.')

### Load and Process PDF documents

#### Create Chunk Contexts for Contextual Retrieval

![](https://i.imgur.com/LRhKHzk.png)

In [30]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [31]:
# create chunk context generation chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


def generate_chunk_context(document, chunk):

    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                        """

    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    agentic_chunk_chain = (prompt_template
                                |
                            chatgpt
                                |
                            StrOutputParser())

    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    return context

In [32]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [33]:
from glob import glob

pdf_files = glob('./rag_docs/*.pdf')
pdf_files

['./rag_docs/attention_paper.pdf',
 './rag_docs/resnet_paper.pdf',
 './rag_docs/cnn_paper.pdf',
 './rag_docs/vision_transformer.pdf']

In [34]:
paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp, chunk_size=3500))

Loading pages: ./rag_docs/attention_paper.pdf
Chunking pages: ./rag_docs/attention_paper.pdf
Generating contextual chunks: ./rag_docs/attention_paper.pdf
Finished processing: ./rag_docs/attention_paper.pdf

Loading pages: ./rag_docs/resnet_paper.pdf
Chunking pages: ./rag_docs/resnet_paper.pdf
Generating contextual chunks: ./rag_docs/resnet_paper.pdf
Finished processing: ./rag_docs/resnet_paper.pdf

Loading pages: ./rag_docs/cnn_paper.pdf
Chunking pages: ./rag_docs/cnn_paper.pdf
Generating contextual chunks: ./rag_docs/cnn_paper.pdf
Finished processing: ./rag_docs/cnn_paper.pdf

Loading pages: ./rag_docs/vision_transformer.pdf
Chunking pages: ./rag_docs/vision_transformer.pdf
Generating contextual chunks: ./rag_docs/vision_transformer.pdf
Finished processing: ./rag_docs/vision_transformer.pdf



In [35]:
len(paper_docs)

79

In [36]:
paper_docs[0]

Document(metadata={'id': '52389109-3a96-4198-b9c6-76e3b207f8b8', 'page': 0, 'source': './rag_docs/attention_paper.pdf', 'title': 'attention_paper.pdf'}, page_content='Focuses on the authorship and permissions related to the research paper "Attention Is All You Need," which introduces the Transformer model for sequence transduction tasks. It highlights the contributions of various authors from Google Brain and Google Research, as well as the paper\'s presentation at the 31st Conference on Neural Information Processing Systems (NIPS 2017).\nProvided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAida

### Combine all document chunks in one list

In [37]:
len(wiki_docs_processed)

1801

In [38]:
total_docs = wiki_docs_processed + paper_docs
len(total_docs)

1880

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [39]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [40]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

In [41]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 5 similar documents based on the user input query

In [42]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

In [43]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [44]:
query = "what is machine learning?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'title': 'Machine learning', 'id': '564928', 'page': 1, 'source': 'Wikipedia'}
Content Brief:


Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Metadata: {'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}
Content Brief:


In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.


Metadata: {'page': 1, 'id': '663523', 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, whic


Metadata: {'source': 'Wikipedia', 'id': '6360', 'page': 1, 'title': 'Artificial intelligence'}
Content Brief:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental facu


Metadata: {'title': 'Artificial neural network', 'id': '44742', 'source': 'Wikipedia', 'page': 1}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [45]:
query = "what is the difference between transformers and vision transformers?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'page': 7, 'title': 'vision_transformer.pdf', 'id': '5f2a682d-4ec5-4dd4-9a64-bc314b90bc67', 'source': './rag_docs/vision_transformer.pdf'}
Content Brief:


Focuses on a controlled scaling study of various models, including Vision Transformers and ResNets, evaluating their transfer performance from the JFT-300M dataset. It highlights the performance versus pre-training cost, revealing that Vision Transformers outperform ResNets in terms of compute efficiency and suggesting potential for further scaling efforts. Additionally, it discusses the performance of hybrid models in comparison to pure Vision Transformers.
Published as a conference paper at ICLR 2021
4.4
SCALING STUDY
We perform a controlled scaling study of different models by evaluating transfer performance from
JFT-300M. In this setting data size does not bottleneck the models’ performances, and we assess
performance versus pre-training cost of each model. The model set includes: 7 ResNets, R50x1,
R50x2 R101x1, R152x1, R152x2, pre-trained for 7 epochs, plus R152x2 and R200x3 pre-trained
for 14 epochs; 6 Vision Transformers, ViT-B/32, B/16, L/32, L/16, pre-trained for 7 epochs, plu


Metadata: {'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf', 'page': 0, 'id': '4cc53577-e83f-4046-84b1-b132bd16b715'}
Content Brief:


Focuses on the introduction of the Vision Transformer (ViT) architecture, which applies a standard Transformer model directly to image classification tasks by treating image patches as tokens. It highlights the limitations of traditional convolutional neural networks (CNNs) in computer vision and presents evidence that ViT can achieve competitive performance on various benchmarks with fewer computational resources when pre-trained on large datasets.
Published as a conference paper at ICLR 2021
AN IMAGE IS WORTH 16X16 WORDS:
TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE
Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,
Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,
Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†
∗equal technical contribution, †equal advising
Google Research, Brain Team
{adosovitskiy, neilhoulsby}@google.com
ABSTRACT
While the Transformer architecture has become the de-facto standard for natural
language 


Metadata: {'id': 'af27625e-37c9-434c-a1d6-549c4e308ab7', 'page': 2, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁ


Metadata: {'id': 'e9cf138e-adc2-42ea-8d24-2ae76a5d032a', 'page': 1, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) in image classification tasks, highlighting its ability to achieve state-of-the-art results when pre-trained on large datasets. It contrasts the inductive biases of convolutional neural networks (CNNs) with the advantages of large-scale training for ViT, demonstrating its effectiveness on various benchmarks, including ImageNet and CIFAR-100. Additionally, it references related work in the field of self-attention and image processing.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the


Metadata: {'title': 'vision_transformer.pdf', 'id': '59947efc-8e64-4097-94b9-8ddd9d25dc65', 'page': 7, 'source': './rag_docs/vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT), highlighting how attention distances vary across layers and the implications of localized attention in hybrid models that incorporate convolutional networks. It also discusses the relationship between attention distance and network depth, indicating that deeper layers attend to semantically relevant regions in images.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁcation (Figure 6).
4.6
SELF-SUPERVISION
Transformers show impressive performance on NLP tasks. However, much of their success stems
not only from their excell

## Build the RAG Pipeline

In [46]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [47]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    rag_prompt_template
      |
    chatgpt
)

In [48]:
from IPython.display import display, Markdown

query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in artificial intelligence. Machine learning focuses on the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. These algorithms follow programmed instructions but can also adapt and improve their performance by building models from sample inputs.

Machine learning is particularly useful in scenarios where designing and programming explicit algorithms is impractical. Some common applications of machine learning include:

- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Within machine learning, there are different types of learning approaches, such as supervised learning, where a function is inferred from labeled training data. In supervised learning, the system learns to produce correct results based on known outcomes from the training data.

Additionally, deep learning is a specialized area of machine learning that primarily utilizes neural networks. Deep learning involves multiple layers of processing, allowing the model to learn increasingly abstract representations of the data. This approach is particularly effective for complex tasks like speech recognition, image understanding, and handwriting recognition.

Overall, machine learning enables computers to improve their performance on tasks through experience, making it a powerful tool in the field of artificial intelligence.

In [49]:
query = "What is a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A CNN, or Convolutional Neural Network, is a specialized type of Artificial Neural Network (ANN) primarily used for image-driven pattern recognition tasks. CNNs are designed to process data with a grid-like topology, such as images, and they excel in recognizing patterns and features within visual data.

### Key Characteristics of CNNs:

1. **Architecture**:
   - CNNs are composed of three main types of layers:
     - **Convolutional Layers**: These layers apply convolution operations to the input, allowing the network to learn spatial hierarchies of features. Each neuron in a convolutional layer is connected to a small region of the input, which helps in detecting local patterns.
     - **Pooling Layers**: These layers reduce the spatial dimensions of the input, helping to decrease the computational load and control overfitting by summarizing the features.
     - **Fully-Connected Layers**: These layers connect every neuron in one layer to every neuron in the next layer, typically used at the end of the network to produce the final output.

2. **Dimensionality**:
   - The neurons in CNNs are organized into three dimensions: height, width, and depth. This structure allows CNNs to effectively process and analyze image data, where height and width correspond to the spatial dimensions of the image, and depth corresponds to the number of channels (e.g., RGB for color images).

3. **Feature Extraction**:
   - CNNs are particularly effective at extracting features from images through the stacking of multiple convolutional layers, which allows for the selection of increasingly complex features as the data passes through the network.

4. **Learning Process**:
   - Similar to traditional ANNs, CNNs learn by adjusting the weights of connections based on the input data and the associated output. They utilize techniques such as backpropagation to optimize their performance.

5. **Applications**:
   - CNNs are widely used in various applications, including image classification, object detection, and facial recognition, due to their ability to automatically learn and encode image-specific features.

In summary, CNNs represent a powerful and efficient architecture for processing visual data, leveraging their unique layer structure to excel in tasks related to image recognition and analysis.

In [50]:
query = "How is a resnet better than a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet (Residual Network) is considered better than a traditional CNN (Convolutional Neural Network) for several reasons, primarily due to its architectural innovations that address optimization challenges associated with deeper networks. Here are the key advantages of ResNets over standard CNNs:

1. **Residual Learning Framework**: ResNets introduce the concept of residual learning, where the network learns to predict the residual (the difference) between the desired output and the input. This is formalized as \( F(x) + x \), where \( F(x) \) is the residual mapping. This approach simplifies the learning process, making it easier for the network to optimize.

2. **Shortcut Connections**: ResNets utilize shortcut connections that skip one or more layers. These connections allow gradients to flow more easily during backpropagation, mitigating issues like vanishing gradients that often occur in very deep networks. As a result, ResNets can be trained effectively even with a significantly increased number of layers.

3. **Improved Training and Validation Performance**: Empirical evidence shows that ResNets achieve lower training and validation errors compared to their plain counterparts. For instance, in experiments on the ImageNet dataset, a 34-layer ResNet outperformed a 34-layer plain network by a notable margin (25.03% vs. 28.54% top-1 error). This indicates that ResNets can generalize better and achieve higher accuracy.

4. **Ability to Train Deeper Networks**: Traditional CNNs often suffer from degradation problems as the depth increases, leading to higher training errors. In contrast, ResNets can maintain or even improve performance as they become deeper, as demonstrated by the successful training of networks with over 100 layers, including the 152-layer ResNet, which achieved state-of-the-art results on various benchmarks.

5. **Lower Computational Complexity**: Despite their increased depth, ResNets can maintain lower computational complexity compared to other architectures like VGG networks. For example, a 152-layer ResNet has lower FLOPs (floating point operations) than VGG-16, making it more efficient.

6. **Generalization Across Tasks**: ResNets have shown strong generalization performance across various recognition tasks, not just in image classification but also in object detection and segmentation, leading to significant performance gains in competitions like ILSVRC and COCO.

In summary, ResNets improve upon traditional CNNs by addressing optimization difficulties, enabling the training of deeper networks, and achieving better performance with lower computational costs.

In [51]:
query = "What is NLP and its relation to linguistics?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Natural Language Processing (NLP) is a field within Artificial Intelligence that focuses on enabling computers to automatically understand and generate human languages. The term "Natural Language" specifically refers to human languages, distinguishing them from programming languages. The overarching goal of NLP is to facilitate seamless interaction between humans and machines through language.

NLP is closely related to linguistics, which is the scientific study of language and its structure. Linguistics provides the foundational theories and frameworks that inform the development of NLP technologies. By leveraging insights from linguistics, NLP aims to improve the accuracy and effectiveness of language understanding and generation tasks performed by computers. This relationship underscores the importance of linguistic principles in the design and implementation of NLP systems.

In [52]:
query = "What is the difference between AI, ML and DL?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The difference between AI, ML, and DL can be summarized as follows:

### Artificial Intelligence (AI)
- **Definition**: AI refers to the ability of a computer program or machine to think and learn, mimicking human cognition. It encompasses a broad range of technologies and applications that enable machines to perform tasks that typically require human intelligence, such as learning, reasoning, and problem-solving.
- **Origin**: The term "Artificial Intelligence" was coined by John McCarthy in 1955.
- **Functionality**: AI systems interpret external data, learn from it, and adapt to achieve specific goals. As technology evolves, tasks once considered to require intelligence (like optical character recognition) may no longer be classified as AI.

### Machine Learning (ML)
- **Definition**: ML is a subfield of AI that focuses on the development of algorithms that allow computers to learn from and make predictions based on data without being explicitly programmed for each task.
- **Functionality**: ML algorithms build models from sample inputs and can make predictions or decisions based on data. It is particularly useful in scenarios where traditional programming is impractical, such as spam filtering and network intrusion detection.

### Deep Learning (DL)
- **Definition**: DL is a specialized subset of machine learning that primarily uses neural networks with multiple layers (known as deep neural networks) to analyze various forms of data.
- **Structure**: Deep learning models often include at least one hidden layer between the input and output layers, allowing for the processing of more abstract representations of data as it passes through these layers.
- **Applications**: DL is particularly effective for complex tasks such as speech recognition, image classification, and understanding handwriting, which are challenging for traditional algorithms.

In summary, AI is the overarching field that includes both ML and DL. ML is a method within AI that enables learning from data, while DL is a more advanced technique within ML that utilizes deep neural networks for processing complex data.

In [53]:
query = "What is the difference between transformers and vision transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The difference between transformers and vision transformers (ViTs) primarily lies in their application and the way they process input data.

1. **Input Representation**:
   - **Transformers**: In traditional transformers, the input consists of sequences of tokens, typically used in natural language processing (NLP). Each token represents a word or a sub-word in a sentence.
   - **Vision Transformers (ViTs)**: ViTs adapt the transformer architecture for image classification tasks by treating image patches as tokens. An image is divided into fixed-size patches, which are then flattened and linearly embedded into a sequence of vectors. This sequence is processed similarly to how words are processed in NLP.

2. **Architecture**:
   - **Transformers**: The standard transformer architecture includes layers of multi-headed self-attention and feed-forward neural networks, designed to capture relationships between tokens in a sequence.
   - **Vision Transformers**: ViTs maintain the core transformer architecture but modify the input to accommodate 2D image data. They utilize position embeddings to retain spatial information about the patches, allowing the model to understand the layout of the image.

3. **Performance and Efficiency**:
   - **Transformers**: In NLP, transformers have become the standard due to their ability to scale and perform well on large datasets, often requiring substantial computational resources.
   - **Vision Transformers**: ViTs have shown competitive performance in image classification tasks, often outperforming traditional convolutional neural networks (CNNs) like ResNets in terms of compute efficiency. They can achieve high accuracy with fewer computational resources when pre-trained on large datasets, such as JFT-300M.

4. **Inductive Bias**:
   - **Transformers**: Traditional transformers do not have inherent inductive biases related to the structure of language, relying on large amounts of data for training.
   - **Vision Transformers**: ViTs lack some of the inductive biases present in CNNs, such as translation equivariance and locality, which can affect their performance on smaller datasets. However, when trained on sufficiently large datasets, ViTs can outperform CNNs.

In summary, while both transformers and vision transformers share a common architectural foundation, they differ significantly in their input data types, processing methods, and performance characteristics in their respective domains.

In [54]:
query = "How is self-attention important in transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Self-attention is crucial in transformers, particularly in the Vision Transformer (ViT), for several reasons:

1. **Integration of Information**: Self-attention allows the ViT to integrate information across the entire image, even in the lowest layers. This capability enables the model to attend to various parts of the image simultaneously, which is essential for understanding complex visual patterns and relationships.

2. **Attention Distance**: The concept of "attention distance" is significant in understanding how self-attention operates within the ViT. It refers to the average distance in image space across which information is integrated based on the attention weights. This attention distance is analogous to the receptive field size in convolutional neural networks (CNNs). The findings indicate that some attention heads can attend to most of the image from the lowest layers, demonstrating the model's ability to utilize global information effectively.

3. **Layer-wise Behavior**: The behavior of attention mechanisms varies across layers. In the lower layers, attention distances are consistently small, indicating highly localized attention. As the network depth increases, the attention distance grows, allowing deeper layers to focus on semantically relevant regions of the image. This progression enhances the model's ability to capture more abstract and meaningful features as it processes the input.

4. **Comparison with Hybrid Models**: The attention mechanism in ViT is less pronounced in hybrid models that incorporate convolutional networks, suggesting that self-attention may serve a similar function to early convolutional layers in CNNs. This highlights the unique role of self-attention in enabling the ViT to achieve state-of-the-art results in image classification tasks, particularly when trained on large datasets.

In summary, self-attention in transformers, especially in the context of the Vision Transformer, is vital for enabling global information integration, adapting attention distances across layers, and enhancing the model's performance in visual tasks.

In [55]:
query = "How does a resnet work?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet, or Residual Network, operates on the principle of residual learning to address the challenges associated with training deep neural networks. Here’s a detailed explanation of how it works:

### Key Concepts of ResNet

1. **Residual Learning**:
   - Instead of learning the desired underlying mapping \( H(x) \) directly, ResNets learn a residual mapping \( F(x) = H(x) - x \). This means that the network is designed to learn the difference between the desired output and the input, which is often easier than learning the output directly.

2. **Shortcut Connections**:
   - ResNets utilize shortcut connections that skip one or more layers. These connections perform identity mapping, allowing the input \( x \) to be added directly to the output of the stacked layers. This can be mathematically represented as \( F(x) + x \).
   - The use of these shortcuts helps to mitigate the degradation problem, where deeper networks tend to perform worse than their shallower counterparts due to optimization difficulties.

3. **Optimization Benefits**:
   - The architecture allows for easier optimization of deeper networks. Empirical evidence shows that ResNets can achieve lower training errors and better generalization as the depth of the network increases, unlike plain networks which suffer from higher training errors with increased depth.

### Architectural Details

- ResNets can be constructed with various depths, such as 18, 34, 50, 101, and 152 layers. Each layer typically consists of convolutional operations followed by batch normalization and ReLU activation functions.
- The architecture includes bottleneck blocks, especially in deeper networks (like the 50, 101, and 152-layer ResNets), which help maintain lower computational complexity while achieving significant accuracy gains.

### Performance

- ResNets have demonstrated superior performance on benchmark datasets like ImageNet and CIFAR-10. For instance, a 152-layer ResNet achieved a top-5 validation error of 4.49% on ImageNet, outperforming previous models and winning the ILSVRC 2015 competition.
- The architecture allows for training extremely deep networks (over 100 layers) effectively, which was previously challenging with traditional deep learning architectures.

### Conclusion

In summary, ResNets leverage the concept of residual learning and shortcut connections to facilitate the training of very deep neural networks, overcoming the optimization challenges that typically arise with increased depth. This innovative approach has led to significant improvements in accuracy and performance across various image recognition tasks.

In [56]:
query = "What is LangGraph?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

In [57]:
query = "What is an Agentic AI System?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The context provided does not contain specific information about an "Agentic AI System." Therefore, I don't know what an Agentic AI System is.

In [58]:
query = "What is LangChain?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

# Build a RAG System with Sources

In [59]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [60]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [61]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
result

{'context': [Document(id='f41be5b7-f487-4157-be22-4a9f1e7abb2a', metadata={'page': 1, 'title': 'Machine learning', 'source': 'Wikipedia', 'id': '564928'}, page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'),
  Document(id='9d1a4805-deb9-43b1-9478-a8e0207cca93', metadata={'source': 'Wikipedia

In [62]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['response']))
    print('='*50)
    print('Sources:')
    for source in result_obj['context']:
        print('Metadata:', source.metadata)
        print('Content Brief:')
        display(Markdown(source.page_content))
        print()


In [63]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is machine learning?


Response:


Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in artificial intelligence. Machine learning focuses on the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. These algorithms follow programmed instructions but can also adapt and improve their performance by building models from sample inputs.

Machine learning is particularly useful in scenarios where designing and programming explicit algorithms is impractical. Some common applications of machine learning include:

- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Within machine learning, there are different approaches, such as supervised learning, where a function is inferred from labeled training data. In this case, the system learns to produce correct results based on known outcomes, typically using vectors for training data and results.

Additionally, deep learning is a specialized area of machine learning that utilizes neural networks, particularly those with multiple layers (known as multi-layer neural networks). Deep learning is effective for complex tasks like speech recognition, image understanding, and handwriting recognition, which are challenging for computers but relatively easy for humans. 

Overall, machine learning represents a significant advancement in the ability of computers to process information and make decisions autonomously, evolving from traditional programming methods.

Sources:
Metadata: {'id': '564928', 'source': 'Wikipedia', 'page': 1, 'title': 'Machine learning'}
Content Brief:


...

Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, which make them incompatible with neuroscience evidences.


Metadata: {'page': 1, 'id': '6360', 'title': 'Artificial intelligence', 'source': 'Wikipedia'}
Content Brief:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental faculties once thought to require intelligence are removed from the definition. For example, optical character recognition is no longer perceived as an example of "artificial intelligence": it is just a routine technology.


Metadata: {'page': 1, 'id': '44742', 'title': 'Artificial neural network', 'source': 'Wikipedia'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [64]:
query = "What is the difference between AI, ML and DL?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is the difference between AI, ML and DL?


Response:


The difference between AI, ML, and DL can be summarized as follows:

### Artificial Intelligence (AI)
- **Definition**: AI refers to the ability of a computer program or machine to think and learn, mimicking human cognition. It encompasses a broad range of technologies and applications aimed at making machines "smart."
- **Scope**: AI includes various subfields, one of which is machine learning. It focuses on systems that can interpret external data, learn from it, and adapt to achieve specific goals.
- **Examples**: AI applications can range from simple tasks like optical character recognition to more complex problem-solving scenarios.

### Machine Learning (ML)
- **Definition**: ML is a subfield of AI that enables computers to learn from data without being explicitly programmed. It involves the study and construction of algorithms that can make predictions or decisions based on input data.
- **Functionality**: ML algorithms build models from sample inputs and can operate in various learning modes, including supervised, unsupervised, and semi-supervised learning.
- **Examples**: Common applications of ML include spam filtering, network intrusion detection, and computer vision tasks.

### Deep Learning (DL)
- **Definition**: DL is a specialized subset of machine learning that primarily uses neural networks with multiple layers (deep neural networks) to process data. It is particularly effective for tasks that require understanding complex patterns in large datasets.
- **Architecture**: Deep learning models often have at least one hidden layer between the input and output layers, allowing them to learn increasingly abstract representations of the data.
- **Examples**: DL is commonly used in applications such as speech recognition, image classification, and natural language processing.

In summary, AI is the overarching field that includes both ML and DL, with ML being a method of achieving AI through data-driven learning, and DL being a more advanced technique within ML that utilizes deep neural networks for complex tasks.

Sources:
Metadata: {'page': 1, 'id': '663523', 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


...

Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental faculties once thought to require intelligence are removed from the definition. For example, optical character recognition is no longer perceived as an example of "artificial intelligence": it is just a routine technology.


Metadata: {'id': '69ccb3c9-9e7c-4dcb-8781-e32a8611be33', 'page': 3, 'title': 'cnn_paper.pdf', 'source': './rag_docs/cnn_paper.pdf'}
Content Brief:


Focuses on the architectural differences of Convolutional Neural Networks (CNNs) compared to traditional Artificial Neural Networks (ANNs), emphasizing the three-dimensional organization of neurons and the specific types of layers that comprise CNNs, including convolutional, pooling, and fully-connected layers. It also outlines the basic functionality of these layers in processing image data for tasks such as classification.
4
Keiron O’Shea et al.
One of the key differences is that the neurons that the layers within the CNN
are comprised of neurons organised into three dimensions, the spatial dimen-
sionality of the input (height and the width) and the depth. The depth does not
refer to the total number of layers within the ANN, but the third dimension of a
activation volume. Unlike standard ANNS, the neurons within any given layer
will only connect to a small region of the layer preceding it.
In practice this would mean that for the example given earlier, the input ’vol-
ume’ will have a dimensionality of 64 × 64 × 3 (height, width and depth), lead-
ing to a ﬁnal output layer comprised of a dimensionality of 1 × 1 × n (where
n represents the possible number of classes) as we would have condensed the
full input dimensionality into a smaller volume of class scores ﬁled across the
depth dimension.
2.1
Overall architecture
CNNs are comprised of three types of layers. These are convolutional layers,
pooling layers and fully-connected layers. When these layers are stacked, a
CNN architecture has been formed. A simpliﬁed CNN architecture for MNIST
classiﬁcation is illustrated in Figure 2.
input
0
9
convolution
 w/ReLu
pooling
output
fully-connected
w/ ReLu
fully-connected
...
Fig. 2: An simple CNN architecture, comprised of just ﬁve layers
The basic functionality of the example CNN above can be broken down into
four key areas.
1. As found in other forms of ANN, the input layer will hold the pixel values
of the image.
2. The convolutional layer will determine the output of neurons of which are
connected to local regions of the input through the calculation of the scalar
product between their weights and the region connected to the input vol-
ume. The rectiﬁed linear unit (commonly shortened to ReLu) aims to apply


Metadata: {'page': 1, 'title': 'Loop AI Labs', 'source': 'Wikipedia', 'id': '669662'}
Content Brief:


Loop AI Labs is an AI and cognitive computing company that focuses on language understanding technology. The company was founded in San Francisco in 2012 by Italian entrepreneur Gianmauro Calafiore, who sold his company Gsmbox to in 2004 and then relocated from Italy to San Francisco. Wanting to start an artificial intelligence company, he recruited two veterans of the project, the largest government-funded AI project in history, who had worked on the project at and Stanford University's . The original company name, "Soshoma", was changed to Loop AI Labs in 2015 after the company decided to change its focus from consumer-oriented to enterprise. Loop AI Labs is headquartered in San Francisco, California, with offices in New York, Milan, and Singapore. The company is privately funded. On May 4, 2017, Loop AI Labs entered into a deal with , a leading European provider of mobile messaging and solutions, to bring their cognitive computing technology to LINK's business clients, which cover 234 million people across Europe.

In [65]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is the difference between transformers and vision transformers?


Response:


The difference between transformers and vision transformers (ViTs) primarily lies in their application and the way they process input data.

1. **Input Representation**:
   - **Transformers**: In traditional transformers, the input is typically a sequence of tokens, such as words in natural language processing (NLP). Each token is represented as an embedding, and the model processes these embeddings using self-attention mechanisms.
   - **Vision Transformers (ViTs)**: ViTs adapt this architecture for image classification tasks by treating image patches as tokens. An image is divided into fixed-size patches, which are then flattened and linearly embedded into a sequence of vectors. This sequence is fed into the transformer model, similar to how word embeddings are processed in NLP.

2. **Architecture**:
   - **Transformers**: The standard transformer architecture consists of layers of multi-headed self-attention and feed-forward neural networks, designed to capture relationships between tokens in a sequence.
   - **Vision Transformers**: ViTs maintain the same basic architecture as standard transformers but are specifically designed to handle the spatial structure of images. They incorporate position embeddings to retain spatial information about the patches, allowing the model to understand the layout of the image.

3. **Performance and Efficiency**:
   - **Transformers**: In NLP, transformers have become the de-facto standard due to their ability to scale and perform well on large datasets.
   - **Vision Transformers**: ViTs have shown competitive performance in image classification tasks, often outperforming traditional convolutional neural networks (CNNs) like ResNets in terms of compute efficiency when pre-trained on large datasets. They require significantly less computational resources to achieve similar or better performance compared to CNNs.

4. **Inductive Bias**:
   - **Transformers**: Traditional transformers do not have inherent inductive biases related to the structure of the data they process.
   - **Vision Transformers**: ViTs lack some of the inductive biases present in CNNs, such as translation equivariance and locality, which can affect their performance on smaller datasets. However, when trained on large datasets, ViTs can leverage their architecture to achieve high accuracy.

In summary, while both transformers and vision transformers share a common architectural foundation, they differ in their input data representation, application domain, and performance characteristics, particularly in how they handle image data compared to sequential text data.

Sources:
Metadata: {'source': './rag_docs/vision_transformer.pdf', 'id': '5f2a682d-4ec5-4dd4-9a64-bc314b90bc67', 'title': 'vision_transformer.pdf', 'page': 7}
Content Brief:


...

Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁcation, we use the standard approach of adding an extra learnable
“classiﬁcation token” to the sequence. The illustration of the Transformer encoder was inspired by
Vaswani et al. (2017).
3
METHOD
In model design we follow the original Transformer (Vaswani et al., 2017) as closely as possible.
An advantage of this intentionally simple setup is that scalable NLP Transformer architectures – and
their efﬁcient implementations – can be used almost out of the box.
3.1
VISION TRANSFORMER (VIT)
An overview of the model is depicted in Figure 1. The standard Transformer receives as input a 1D
sequence of token embeddings. To handle 2D images, we reshape the image x ∈RH×W ×C into a
sequence of ﬂattened 2D patches xp ∈RN×(P 2·C), where (H, W) is the resolution of the original
image, C is the number of channels, (P, P) is the resolution of each image patch, and N = HW/P 2
is the resulting number of patches, which also serves as the effective input sequence length for the
Transformer. The Transformer uses constant latent vector size D through all of its layers, so we
ﬂatten the patches and map to D dimensions with a trainable linear projection (Eq. 1). We refer to
the output of this projection as the patch embeddings.
Similar to BERT’s [class] token, we prepend a learnable embedding to the sequence of embed-
ded patches (z0
0 = xclass), whose state at the output of the Transformer encoder (z0
L) serves as the
image representation y (Eq. 4). Both during pre-training and ﬁne-tuning, a classiﬁcation head is at-
tached to z0
L. The classiﬁcation head is implemented by a MLP with one hidden layer at pre-training
time and by a single linear layer at ﬁne-tuning time.
Position embeddings are added to the patch embeddings to retain positional information. We use
standard learnable 1D position embeddings, since we have not observed signiﬁcant performance
gains from using more advanced 2D-aware position embeddings (Appendix D.4). The resulting
sequence of embedding vectors serves as input to the encoder.
The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded self-
attention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before
every block, and residual connections after every block (Wang et al., 2019; Baevski & Auli, 2019).
3


Metadata: {'title': 'vision_transformer.pdf', 'source': './rag_docs/vision_transformer.pdf', 'page': 1, 'id': 'e9cf138e-adc2-42ea-8d24-2ae76a5d032a'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) in image classification tasks, highlighting its ability to achieve state-of-the-art results when pre-trained on large datasets. It contrasts the inductive biases of convolutional neural networks (CNNs) with the advantages of large-scale training for ViT, demonstrating its effectiveness on various benchmarks, including ImageNet and CIFAR-100. Additionally, it references related work in the field of self-attention and image processing.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT approaches
or beats state of the art on multiple image recognition benchmarks. In particular, the best model
reaches the accuracy of 88.55% on ImageNet, 90.72% on ImageNet-ReaL, 94.55% on CIFAR-100,
and 77.63% on the VTAB suite of 19 tasks.
2
RELATED WORK
Transformers were proposed by Vaswani et al. (2017) for machine translation, and have since be-
come the state of the art method in many NLP tasks. Large Transformer-based models are often
pre-trained on large corpora and then ﬁne-tuned for the task at hand: BERT (Devlin et al., 2019)
uses a denoising self-supervised pre-training task, while the GPT line of work uses language mod-
eling as its pre-training task (Radford et al., 2018; 2019; Brown et al., 2020).
Naive application of self-attention to images would require that each pixel attends to every other
pixel. With quadratic cost in the number of pixels, this does not scale to realistic input sizes. Thus,
to apply Transformers in the context of image processing, several approximations have been tried in
the past. Parmar et al. (2018) applied the self-attention only in local neighborhoods for each query
pixel instead of globally. Such local multi-head dot-product self attention blocks can completely
replace convolutions (Hu et al., 2019; Ramachandran et al., 2019; Zhao et al., 2020). In a different
line of work, Sparse Transformers (Child et al., 2019) employ scalable approximations to global self-
attention in order to be applicable to images. An alternative way to scale attention is to apply it in
blocks of varying sizes (Weissenborn et al., 2019), in the extreme case only along individual axes (Ho
et al., 2019; Wang et al., 2020a). Many of these specialized attention architectures demonstrate
promising results on computer vision tasks, but require complex engineering to be implemented
efﬁciently on hardware accelerators.
Most related to ours is the model of Cordonnier et al. (2020), which extracts patches of size 2 × 2
from the input image and applies full self-attention on top. This model is very similar to ViT,
but our work goes further to demonstrate that large scale pre-training makes vanilla transformers
competitive with (or even better than) state-of-the-art CNNs. Moreover, Cordonnier et al. (2020)
use a small patch size of 2 × 2 pixels, which makes the model applicable only to small-resolution
images, while we handle medium-resolution images as well.
There has also been a lot of interest in combining convolutional neural networks (CNNs) with forms
of self-attention, e.g. by augmenting feature maps for image classiﬁcation (Bello et al., 2019) or by
further processing the output of a CNN using self-attention, e.g. for object detection (Hu et al., 2018;
Carion et al., 2020), video processing (Wang et al., 2018; Sun et al., 2019), image classiﬁcation (Wu
et al., 2020), unsupervised object discovery (Locatello et al., 2020), or uniﬁed text-vision tasks (Chen


Metadata: {'page': 7, 'title': 'vision_transformer.pdf', 'id': '59947efc-8e64-4097-94b9-8ddd9d25dc65', 'source': './rag_docs/vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT), highlighting how attention distances vary across layers and the implications of localized attention in hybrid models that incorporate convolutional networks. It also discusses the relationship between attention distance and network depth, indicating that deeper layers attend to semantically relevant regions in images.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁcation (Figure 6).
4.6
SELF-SUPERVISION
Transformers show impressive performance on NLP tasks. However, much of their success stems
not only from their excellent scalability but also from large scale self-supervised pre-training (Devlin
8

In [66]:
query = "What is an Agentic AI System?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is an Agentic AI System?


Response:


The context provided does not contain specific information about an "Agentic AI System." Therefore, I don't know what an Agentic AI System is.

Sources:
Metadata: {'source': 'Wikipedia', 'page': 1, 'title': 'Artificial intelligence', 'id': '6360'}
Content Brief:


...

Swarm Intelligence is a field of Computer science. It is a form of Artificial intelligence. Some animals, mostly insects like ants, or bees form large colonies. These colonies are made of many animals that communicate with each other. Each animal is relatively simple, but by working together with other animals it is able to solve complex tasks. Swarm intelligence wants to obtain similar behaviour than that observed with these animals. Instead of the animals, so called "agents" are used.


Metadata: {'title': 'Shakey the robot', 'page': 1, 'source': 'Wikipedia', 'id': '692745'}
Content Brief:


Shakey the Robot was the first general purpose mobile AI robot. The project combined research in robotics, computer vision, and natural language processing. Because of this, it was the first project that melded logical reasoning and physical action. Shakey was developed at the Artificial Intelligence Center of Stanford Research Institute (now called SRI International) by Nils John Nilsson from 1966 to 1972.


Metadata: {'page': 1, 'title': 'Robot lawyer', 'source': 'Wikipedia', 'id': '564218'}
Content Brief:


A robot lawyer is an artificial intelligence (AI) computer program. It is designed to ask the same questions as a real lawyer about certain legal issues. Robot lawyers are being used in many countries around the world including the United States, the United Kingdom, and Holland.

# Build a RAG System with Source Citations Agentic Pipeline

In [67]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)
rag_prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            


In [68]:
citations_prompt = """You are an assistant who is an expert in analyzing answers to questions
                      and finding out referenced citations from context articles.

                      Given the following question, context and generated answer,
                      analyze the generated answer and quote citations from context articles
                      that can be used to justify the generated answer.

                      Question:
                      {question}

                      Context Articles:
                      {context}

                      Answer:
                      {answer}
                  """

cite_prompt_template = ChatPromptTemplate.from_template(citations_prompt)
cite_prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant who is an expert in analyzing answers to questions
                      and finding out referenced citations from context articles.

                      Given the following question, context and generated answer,
                      analyze the generated answer and quote citations from context articles
                      that can be used to justify the generated answer.

                      Question:
                      {question}

                      Context Articles:
                      {context}

                      Answer:
                      {answer}
                  


In [69]:
from pydantic import BaseModel, Field
from typing import List

class Citation(BaseModel):
    id: str = Field(description="""The string ID of a SPECIFIC context article
                                   which justifies the answer.""")
    source: str = Field(description="""The source of the SPECIFIC context article
                                       which justifies the answer.""")
    title: str = Field(description="""The title of the SPECIFIC context article
                                      which justifies the answer.""")
    page: int = Field(description="""The page number of the SPECIFIC context article
                                     which justifies the answer.""")
    quotes: str = Field(description="""The VERBATIM sentences from the SPECIFIC context article
                                      that are used to generate the answer.
                                      Should be exact sentences from context article without missing words.""")


class QuotedCitations(BaseModel):
    """Quote citations from given context articles
       that can be used to justify the generated answer. Can be multiple articles."""
    citations: List[Citation] = Field(description="""Citations (can be multiple) from the given
                                                     context articles that justify the answer.""")

In [70]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
structured_chatgpt = chatgpt.with_structured_output(QuotedCitations)


def format_docs_with_metadata(docs: List[Document]) -> str:
    formatted_docs = [
        f"""Context Article ID: {doc.metadata['id']}
            Context Article Source: {doc.metadata['source']}
            Context Article Title: {doc.metadata['title']}
            Context Article Page: {doc.metadata['page']}
            Context Article Details: {doc.page_content}
         """
            for i, doc in enumerate(docs)
    ]
    return "\n\n" + "\n\n".join(formatted_docs)

rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs_with_metadata)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

cite_response_chain = (
    {
        "context": itemgetter('context'),
        "question": itemgetter("question"),
        "answer": itemgetter("answer")
    }
        |
    cite_prompt_template
        |
    structured_chatgpt
)

rag_chain_w_citations = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(answer=rag_response_chain)
        |
    RunnablePassthrough.assign(citations=cite_response_chain)
)

In [71]:
query = "What is machine learning"
result = rag_chain_w_citations.invoke(query)
result

{'context': [Document(id='f41be5b7-f487-4157-be22-4a9f1e7abb2a', metadata={'id': '564928', 'title': 'Machine learning', 'source': 'Wikipedia', 'page': 1}, page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'),
  Document(id='9d1a4805-deb9-43b1-9478-a8e0207cca93', metadata={'id': '359370', 'pag

In [72]:
result['citations'].dict()['citations']

/tmp/ipython-input-72-3486563740.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result['citations'].dict()['citations']


[{'id': '564928',
  'source': 'Wikipedia',
  'title': 'Machine learning',
  'page': 1,
  'quotes': 'Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'}]

In [73]:
import re
# used mostly for nice display formatting, ignore if not needed
def get_cited_context(result_obj):
    # Dictionary to hold separate citation information for each unique source and title combination
    source_with_citations = {}

    def highlight_text(context, quote):
        # Normalize whitespace and remove unnecessary punctuation
        quote = re.sub(r'\s+', ' ', quote).strip()
        context = re.sub(r'\s+', ' ', context).strip()

        # Split quote into phrases, being careful with punctuation
        phrases = [phrase.strip() for phrase in re.split(r'[.!?]', quote) if phrase.strip()]

        highlighted_context = context

        for phrase in phrases: # for each quoted phrase

            # Create regex pattern to match cited phrases
            # Escape special regex characters, but preserve word boundaries
            escaped_phrase = re.escape(phrase)
            # Create regex pattern that allows for slight variations
            pattern = re.compile(r'\b' + escaped_phrase + r'\b', re.IGNORECASE)

            # Replace all matched phrases with bolded version
            highlighted_context = pattern.sub(lambda m: f"**{m.group(0)}**", highlighted_context)

        return highlighted_context

    # Process the citation data
    for cite in result_obj['citations'].dict()['citations']:
        cite_id = cite['id']
        title = cite['title']
        source = cite['source']
        page = cite['page']
        quote = cite['quotes']

        # Check if the (source, title) key exists, and initialize if it doesn't
        if (source, title) not in source_with_citations:
            source_with_citations[(source, title)] = {
                'title': title,
                'source': source,
                'citations': []
            }

        # Find or create the citation entry for this unique (id, page) combination
        citation_entry = next(
            (c for c in source_with_citations[(source, title)]['citations'] if c['id'] == cite_id and c['page'] == page),
            None
        )
        if citation_entry is None:
            citation_entry = {'id': cite_id, 'page': page, 'quote': [quote], 'context': None}
            source_with_citations[(source, title)]['citations'].append(citation_entry)
        else:
            citation_entry['quote'].append(quote)

    # Process context data
    for context in result_obj['context']:
        context_id = context.metadata['id']
        context_page = context.metadata['page']
        source = context.metadata['source']
        title = context.metadata['title']
        page_content = context.page_content

        # Match the context to the correct citation entry by source, title, id, and page
        if (source, title) in source_with_citations:
            for citation in source_with_citations[(source, title)]['citations']:
                if citation['id'] == context_id and citation['page'] == context_page:
                    # Apply highlighting for each quote in the citation's quote list
                    highlighted_content = page_content
                    for quote in citation['quote']:
                        highlighted_content = highlight_text(highlighted_content, quote)
                    citation['context'] = highlighted_content

    # Convert the dictionary to a list of dictionaries for separate entries
    final_result_list = [
        {
            'title': details['title'],
            'source': details['source'],
            'citations': details['citations']
        }
        for details in source_with_citations.values()
    ]

    return final_result_list


In [74]:
get_cited_context(result)

/tmp/ipython-input-73-3913888139.py:31: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for cite in result_obj['citations'].dict()['citations']:


[{'title': 'Machine learning',
  'source': 'Wikipedia',
  'citations': [{'id': '564928',
    'page': 1,
    'quote': ['Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'],
    'context': 'Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It 

In [75]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['answer']))
    print('='*50)
    print('Sources:')
    cited_context = get_cited_context(result_obj)
    for source in cited_context:
        print('Title:', source['title'], ' ', 'Source:', source['source'])
        print('Citations:')
        for citation in source['citations']:
            print('ID:', citation['id'], ' ', 'Page:', citation['page'])
            print('Cited Quotes:')
            display(Markdown('*'+' '.join(citation['quote'])+'*'))
            print('Cited Context:')
            display(Markdown(citation['context']))
            print()


In [76]:
display_results(result)

Query:


What is machine learning


Response:


Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in the broader field of artificial intelligence (AI). 

Machine learning involves the study and construction of algorithms that can learn from and make predictions based on data. These algorithms follow programmed instructions but can also make decisions or predictions based on the data they process. The process typically involves building a model from sample inputs, which allows the system to operate in scenarios where designing and programming explicit algorithms is not feasible.

Some common applications of machine learning include:
- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Overall, machine learning enables systems to improve their performance on tasks over time as they are exposed to more data, making it a powerful tool in various domains.

Sources:
Title: Machine learning   Source: Wikipedia
Citations:
ID: 564928   Page: 1
Cited Quotes:


/tmp/ipython-input-73-3913888139.py:31: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for cite in result_obj['citations'].dict()['citations']:


*Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.*

Cited Context:


Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial intelligence**. **Machine learning explores the study and construction of algorithms which can learn and make predictions on data**. **Such algorithms follow programmed instructions, but can also make predictions or decisions based on data**. **They build a model from sample inputs**. **Machine learning is done where designing and programming explicit algorithms cannot be done**. **Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision**.

In [77]:
query = "What is AI, ML and DL?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


What is AI, ML and DL?


Response:


**Artificial Intelligence (AI)**: AI is defined as the ability of a computer program or machine to think and learn. It is also a field of study aimed at making computers "smart," allowing them to operate independently without being explicitly programmed with commands. The term was coined by John McCarthy in 1955. AI encompasses systems that can interpret external data, learn from it, and adapt to achieve specific goals. As technology advances, tasks once considered to require intelligence, such as optical character recognition, are no longer classified as AI but rather as routine technologies.

**Machine Learning (ML)**: ML is a subfield of computer science that provides computers the ability to learn from data without being explicitly programmed. This concept emerged from the broader field of artificial intelligence. Machine learning involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. It is particularly useful in scenarios where traditional programming is impractical. Examples of machine learning applications include spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.

**Deep Learning (DL)**: DL is a specialized form of machine learning that primarily utilizes neural networks, particularly those with multiple layers (known as multi-layer neural networks). It can involve unsupervised, semi-supervised, or supervised learning sessions. Deep learning is particularly effective for complex tasks such as speech recognition, image understanding, and handwriting recognition, which are challenging for computers. The architecture of deep learning models is inspired by the information processing patterns of biological nervous systems, although they differ significantly from the structural and functional properties of human brains.

Sources:
Title: Artificial intelligence   Source: Wikipedia
Citations:
ID: 6360   Page: 1
Cited Quotes:


...

Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial intelligence**. **Machine learning explores the study and construction of algorithms which can learn and make predictions on data**. **Such algorithms follow programmed instructions, but can also make predictions or decisions based on data**. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Title: Deep learning   Source: Wikipedia
Citations:
ID: 663523   Page: 1
Cited Quotes:


*Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer.*

Cited Context:


**Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks**. **As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised**. **In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer**. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, which make them incompatible with neuroscience evidences.

In [78]:
query = "How is Machine learning related to supervised learning and clustering?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


How is Machine learning related to supervised learning and clustering?


Response:


Machine learning is a broad field that encompasses various techniques and methodologies for enabling computers to learn from data. Within this field, two important concepts are supervised learning and clustering.

### Supervised Learning
Supervised learning is a specific type of machine learning where the algorithm is trained on labeled data. This means that the training dataset includes both the input data and the corresponding correct outputs (labels). The primary goal of supervised learning is to infer a function that can map inputs to the correct outputs based on the training data. The system learns to produce a "classifier" that can make predictions on new, unseen data by generalizing from the training examples. This approach relies on inductive reasoning to derive patterns from the labeled data.

### Clustering
Clustering, or cluster analysis, is another technique within the realm of machine learning, but it falls under the category of unsupervised learning. In clustering, the objective is to group a set of objects in such a way that objects within the same group (or cluster) are more similar to each other than to those in other groups. Unlike supervised learning, clustering does not use labeled data; instead, it seeks to identify inherent structures or patterns in the data based solely on the features of the objects being analyzed.

### Relationship Between Machine Learning, Supervised Learning, and Clustering
- **Machine Learning**: This is the overarching field that includes various learning paradigms, including both supervised and unsupervised learning techniques.
- **Supervised Learning**: A subset of machine learning focused on learning from labeled data to make predictions or classifications.
- **Clustering**: A technique within machine learning that involves grouping data without prior labels, focusing on the similarities among data points.

In summary, supervised learning and clustering are both integral parts of machine learning, each serving different purposes and utilizing different types of data. Supervised learning relies on labeled data to train models for prediction, while clustering seeks to discover patterns in unlabeled data by grouping similar items together.

Sources:
Title: Supervised learning   Source: Wikipedia
Citations:
ID: 359370   Page: 1
Cited Quotes:


...

**Machine learning gives computers the ability to learn without being explicitly programmed** (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial intelligence**. **Machine learning explores the study and construction of algorithms which can learn and make predictions on data**. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Title: Cluster analysis   Source: Wikipedia
Citations:
ID: 593732   Page: 1
Cited Quotes:


*Clustering or cluster analysis is a type of data analysis. The analyst groups objects so that objects in the same group (called a cluster) are more similar to each other than to objects in other groups (clusters) in some way.*

Cited Context:


**Clustering or cluster analysis is a type of data analysis**. **The analyst groups objects so that objects in the same group (called a cluster) are more similar to each other than to objects in other groups (clusters) in some way**. This is a common task in data mining.

In [79]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


What is the difference between transformers and vision transformers?


Response:


The difference between traditional Transformers and Vision Transformers (ViTs) primarily lies in their application and the way they process input data.

### Traditional Transformers
- **Application**: Originally designed for natural language processing (NLP) tasks, traditional Transformers operate on sequences of tokens, such as words in a sentence.
- **Input Representation**: They take a sequence of embeddings as input, where each token (word) is represented by a vector. The architecture relies heavily on self-attention mechanisms to capture relationships between tokens in the sequence.
- **Inductive Bias**: Transformers do not inherently possess inductive biases like locality or translation equivariance, which are common in convolutional neural networks (CNNs). This can lead to challenges when applied to tasks with limited data.

### Vision Transformers (ViTs)
- **Application**: ViTs adapt the Transformer architecture for image classification tasks by treating image patches as tokens. This allows them to directly apply the Transformer model to visual data.
- **Input Representation**: Images are divided into fixed-size patches, which are then flattened and linearly embedded into a sequence of vectors. These patch embeddings are treated similarly to word embeddings in NLP.
- **Position Embeddings**: ViTs incorporate position embeddings to retain spatial information about the arrangement of patches, which is crucial for understanding the structure of images.
- **Performance**: When pre-trained on large datasets, ViTs can achieve competitive performance on various image recognition benchmarks, often outperforming traditional CNNs in terms of compute efficiency. They require fewer computational resources to achieve similar or better results compared to CNNs.

### Summary
In essence, while traditional Transformers are optimized for sequential data like text, Vision Transformers extend this architecture to handle image data by treating image patches as sequences of tokens, thus leveraging the strengths of self-attention in a visual context.

Sources:
Title: vision_transformer.pdf   Source: ./rag_docs/vision_transformer.pdf
Citations:
ID: 5f2a682d-4ec5-4dd4-9a64-bc314b90bc67   Page: 7
Cited Quotes:


...

Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture. Published as a conference paper at ICLR 2021 Transformer Encoder MLP Head Vision Transformer (ViT) * Linear Projection of Flattened Patches * Extra learnable [ cl ass] embedding 1 2 3 4 5 6 7 8 9 0 Patch + Position Embedding Class Bird Ball Car ... Embedded Patches Multi-Head Attention Norm MLP Norm + L x + Transformer Encoder Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them, add position embeddings, and feed the resulting sequence of vectors to a standard Transformer encoder. In order to perform classiﬁcation, we use the standard approach of adding an extra learnable “classiﬁcation token” to the sequence. The illustration of the Transformer encoder was inspired by Vaswani et al. (2017). 3 METHOD In model design we follow the original Transformer (Vaswani et al., 2017) as closely as possible. An advantage of this intentionally simple setup is that scalable NLP Transformer architectures – and their efﬁcient implementations – can be used almost out of the box. 3.1 VISION TRANSFORMER (VIT) An overview of the model is depicted in Figure 1. The standard Transformer receives as input a 1D sequence of token embeddings. To handle 2D images, we reshape the image x ∈RH×W ×C into a sequence of ﬂattened 2D patches xp ∈RN×(P 2·C), where (H, W) is the resolution of the original image, C is the number of channels, (P, P) is the resolution of each image patch, and N = HW/P 2 is the resulting number of patches, which also serves as the effective input sequence length for the Transformer. The Transformer uses constant latent vector size D through all of its layers, so we ﬂatten the patches and map to D dimensions with a trainable linear projection (Eq. 1). We refer to the output of this projection as the patch embeddings. Similar to BERT’s [class] token, we prepend a learnable embedding to the sequence of embed- ded patches (z0 0 = xclass), whose state at the output of the Transformer encoder (z0 L) serves as the image representation y (Eq. 4). Both during pre-training and ﬁne-tuning, a classiﬁcation head is at- tached to z0 L. The classiﬁcation head is implemented by a MLP with one hidden layer at pre-training time and by a single linear layer at ﬁne-tuning time. Position embeddings are added to the patch embeddings to retain positional information. We use standard learnable 1D position embeddings, since we have not observed signiﬁcant performance gains from using more advanced 2D-aware position embeddings (Appendix D.4). The resulting sequence of embedding vectors serves as input to the encoder. The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded self- attention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before every block, and residual connections after every block (Wang et al., 2019; Baevski & Auli, 2019). 3


ID: e9cf138e-adc2-42ea-8d24-2ae76a5d032a   Page: 1
Cited Quotes:


*However, the picture changes if the models are trained on larger datasets (14M-300M images). We find that large scale training trumps inductive bias.*

Cited Context:


Focuses on the performance of the Vision Transformer (ViT) in image classification tasks, highlighting its ability to achieve state-of-the-art results when pre-trained on large datasets. It contrasts the inductive biases of convolutional neural networks (CNNs) with the advantages of large-scale training for ViT, demonstrating its effectiveness on various benchmarks, including ImageNet and CIFAR-100. Additionally, it references related work in the field of self-attention and image processing. Published as a conference paper at ICLR 2021 inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well when trained on insufﬁcient amounts of data. However, the picture changes if the models are trained on larger datasets (14M-300M images). We ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT approaches or beats state of the art on multiple image recognition benchmarks. In particular, the best model reaches the accuracy of 88.55% on ImageNet, 90.72% on ImageNet-ReaL, 94.55% on CIFAR-100, and 77.63% on the VTAB suite of 19 tasks. 2 RELATED WORK Transformers were proposed by Vaswani et al. (2017) for machine translation, and have since be- come the state of the art method in many NLP tasks. Large Transformer-based models are often pre-trained on large corpora and then ﬁne-tuned for the task at hand: BERT (Devlin et al., 2019) uses a denoising self-supervised pre-training task, while the GPT line of work uses language mod- eling as its pre-training task (Radford et al., 2018; 2019; Brown et al., 2020). Naive application of self-attention to images would require that each pixel attends to every other pixel. With quadratic cost in the number of pixels, this does not scale to realistic input sizes. Thus, to apply Transformers in the context of image processing, several approximations have been tried in the past. Parmar et al. (2018) applied the self-attention only in local neighborhoods for each query pixel instead of globally. Such local multi-head dot-product self attention blocks can completely replace convolutions (Hu et al., 2019; Ramachandran et al., 2019; Zhao et al., 2020). In a different line of work, Sparse Transformers (Child et al., 2019) employ scalable approximations to global self- attention in order to be applicable to images. An alternative way to scale attention is to apply it in blocks of varying sizes (Weissenborn et al., 2019), in the extreme case only along individual axes (Ho et al., 2019; Wang et al., 2020a). Many of these specialized attention architectures demonstrate promising results on computer vision tasks, but require complex engineering to be implemented efﬁciently on hardware accelerators. Most related to ours is the model of Cordonnier et al. (2020), which extracts patches of size 2 × 2 from the input image and applies full self-attention on top. This model is very similar to ViT, but our work goes further to demonstrate that large scale pre-training makes vanilla transformers competitive with (or even better than) state-of-the-art CNNs. Moreover, Cordonnier et al. (2020) use a small patch size of 2 × 2 pixels, which makes the model applicable only to small-resolution images, while we handle medium-resolution images as well. There has also been a lot of interest in combining convolutional neural networks (CNNs) with forms of self-attention, e.g. by augmenting feature maps for image classiﬁcation (Bello et al., 2019) or by further processing the output of a CNN using self-attention, e.g. for object detection (Hu et al., 2018; Carion et al., 2020), video processing (Wang et al., 2018; Sun et al., 2019), image classiﬁcation (Wu et al., 2020), unsupervised object discovery (Locatello et al., 2020), or uniﬁed text-vision tasks (Chen